# Shallow Moment Tutorial (Minimal)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import jax
from jax import numpy as jnp
import pytest
from types import SimpleNamespace
from sympy import cos, pi, Piecewise

from library.zoomy_core.fvm.solver_jax import HyperbolicSolver
from library.zoomy_core.fvm.ode import RK1
import library.zoomy_core.fvm.reconstruction as recon
import library.zoomy_core.fvm.timestepping as timestepping
import library.zoomy_core.fvm.flux as flux
import library.zoomy_core.fvm.nonconservative_flux as nc_flux
from library.zoomy_core.model.boundary_conditions import BoundaryCondition
from library.zoomy_core.model.models.basisfunctions import Basisfunction, Legendre_shifted
from library.zoomy_core.model.models.basismatrices import Basismatrices
from library.zoomy_core.model.models.shallow_moments import ShallowMoments2d
import library.zoomy_core.misc.io as io

import library.zoomy_core.model.initial_conditions as IC
import library.zoomy_core.model.boundary_conditions as BC
import library.zoomy_core.misc.io as io
from library.zoomy_core.mesh.mesh import compute_derivatives
from tests.pdesoft import plots_paper

from library.zoomy_core.misc.misc import Zstruct, Settings


import library.zoomy_core.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing
from library.zoomy_core.mesh.mesh import convert_mesh_to_jax
import argparse


2025-08-08 06:20:40.500 | WARNING  | library.core.fvm.solver_jax:<module>:28 - No module named 'precice'


## Model

In [2]:
level = 0
offset = 1 + level
settings = Settings(
    name="test",
    output=Zstruct(
        directory=f"outputs/junction_{level}", filename="test", output_snapshots=30, clean_directory=True
    ),
)


2025-08-08 06:20:41.268 | WARNING  | library.core.misc.misc:__init__:136 - No 'solver' Zstruct found in Settings. Default: empty Zstruct
2025-08-08 06:20:41.269 | WARNING  | library.core.misc.misc:__init__:139 - No 'model' Zstruct found in Settings. Default: empty Zstruct
2025-08-08 06:20:41.269 | WARNING  | library.core.misc.misc:__init__:156 - No 'parameters' attribute found in model Zstruct. Default: empty Zstruct


In [3]:
inflow_dict = {
    0: lambda t, x, dx, q, qaux, p, n: Piecewise((0.1, t < 0.2), (q[0], True)),
    1: lambda t, x, dx, q, qaux, p, n: Piecewise((-0.3, t < 0.2), (-q[1], True)),
}
inflow_dict.update({1 + i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level)})
inflow_dict.update(
    {1 + offset + i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level + 1)}
)

bcs = BC.BoundaryConditions(
    [
        BC.Lambda(physical_tag="inflow", prescribe_fields=inflow_dict),
        BC.Wall(physical_tag="wall"),
    ]
)


def custom_ic(x):
    Q = np.zeros(3 + 2 * level, dtype=float)
    Q[0] = 0.01
    return Q


ic = IC.UserFunction(custom_ic)

model = ShallowMoments2d(
    level = level,
    boundary_conditions=bcs,
)

main_dir = os.getenv("ZOOMY_DIR")
mesh = petscMesh.Mesh.from_gmsh(
    os.path.join(main_dir, "meshes/channel_junction/mesh_2d_coarse.msh")
    # os.path.join(main_dir, "meshes/channel_junction/mesh_2d_fine.msh")
)


class SMESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1] / Q[0], mesh, derivatives_multi_index=[[0, 0]])[
            :, 0
        ]
        dvdy = compute_derivatives(
            Q[1 + offset] / Q[0], mesh, derivatives_multi_index=[[0, 1]]
        )[:, 0]
        Qaux = Qaux.at[0].set(dudx)
        Qaux = Qaux.at[1].set(dvdy)
        return Qaux


solver = SMESolver(settings=settings)


## Solver

In [4]:
Qnew, Qaux = solver.solve(mesh, model)

io.generate_vtk(
    os.path.join(settings.output.directory, f"{settings.output.filename}.h5")
)

2025-08-08 06:20:46.419 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 1, time: 0.000677, dt: 0.000677, next write at time: 0.011111
2025-08-08 06:20:46.434 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 2, time: 0.001209, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:20:46.459 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 3, time: 0.001740, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:20:46.478 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 4, time: 0.002272, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:20:46.491 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 5, time: 0.002804, dt: 0.000532, next write at time: 0.011111
2025-08-08 06:20:46.508 | INFO     | library.core.fvm.solver_jax:log_callback_hyperbolic:43 - iteration: 6, time: 0.003335, dt: 0.000532, next write at time: 0.01111

## Visualization

In [5]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.name}.h5"))
postprocessing.vtk_project_2d_to_3d(model, settings, Nz=20, filename='out_3d')

2025-08-08 06:20:54.409 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 0/10
2025-08-08 06:20:54.440 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 1/10
2025-08-08 06:20:54.477 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 2/10
2025-08-08 06:20:54.509 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 3/10
2025-08-08 06:20:54.540 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 4/10
2025-08-08 06:20:54.564 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 5/10
2025-08-08 06:20:54.589 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 6/10
2025-08-08 06:20:54.613 | INFO     | library.postprocessing.postprocessing:vtk_project_2d_to_3d:62 - Converted snapshot 7/10
